In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import precision_score, recall_score, f1_score
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize the MNIST dataset
])

In [ ]:
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)


Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9912422/9912422 [00:01<00:00, 5025306.35it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28881/28881 [00:00<00:00, 133755.82it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1648877/1648877 [00:06<00:00, 243520.71it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4542/4542 [00:00<00:00, 3027257.07it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
     if torch.backends.mps.is_available()
    else "cpu"
)

In [ ]:
def train_and_test(model, train_loader, test_loader, criterion, optimizer, num_epochs, device):
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

  train_loss_list = []


  for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
      images = images.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
    scheduler.step()
  # Generating Loss Curve
    train_loss = running_loss / len(train_loader)
    train_loss_list.append({'train_loss':train_loss})
    print(f'[Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')

  # Test the model

  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for data in test_loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  # Calculating Accuracy
  accuracy = 100 * correct / total

  #Evaluation Metrics
  precision = precision_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='macro')
  recall = recall_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='macro')
  f1 = f1_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='macro')

  print(f'Accuracy of the network : {accuracy:.2f}%')
  print(f'Precision: {precision:.2f}')
  print(f'Recall: {recall:.2f}')
  print(f'F1-score: {f1:.2f}')

  metrics = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1}
  return metrics, train_loss_list

In [ ]:
class Lenet_5(nn.Module):
  def __init__(self, num_classes=10):
    super(Lenet_5, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
    self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
    self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
    self.fc1 = nn.Linear(16*4*4, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self,x):
    x = self.pool(torch.relu(self.conv1(x)))
    x = self.pool(torch.relu(self.conv2(x)))
    x = x.view(-1, 16*4*4)
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
class BasicBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride=1):
    super(BasicBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(out_channels)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_channels != out_channels:
      self.shortcut = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                                    nn.BatchNorm2d(out_channels))
  def forward(self,x):
    out = torch.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    out += self.shortcut(x)
    out = torch.relu(out)
    return out

class ResNet(nn.Module):
  def __init__(self, block, num_blocks, num_classes=10):
    super(ResNet, self).__init__()
    self.in_channels = 64
    self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias = False)
    self.bn1 = nn.BatchNorm2d(64)
    self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
    self.fc = nn.Linear(512, num_classes)

  def  _make_layer(self,block,out_channels, num_blocks, stride):
    strides = [stride] + [1]*(num_blocks-1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_channels, out_channels, stride))
      self.in_channels = out_channels
    return nn.Sequential(*layers)

  def forward(self,x):
    out = torch.relu(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = torch.mean(out, dim=[2,3])
    out = self.fc(out)
    return out
def ResNet18():
  return ResNet(BasicBlock, [2, 2, 2, 2])

net = ResNet18()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
class Alexnet(nn.Module):
  def __init__(self, num_classes=10):
    super(Alexnet, self).__init__()
    self.features = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(64, 192, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(192, 384, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )
    self.classifier = nn.Sequential(
        nn.Dropout(),
        nn.Linear(256*3*3, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096, num_classes),
    )
  def forward(self,x):
    x = self.features(x)
    x = x.view(x.size(0), 256*3*3)
    x = self.classifier(x)
    return x

In [ ]:

class VGGnet(nn.Module):
    def __init__(self, num_classes=10):
        super(VGGnet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        self.conv5 = nn.Conv2d(128,256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(256*3*3, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)

    def forward(self,x):
      x = torch.relu(self.conv1(x))
      x = torch.relu(self.conv2(x))
      x = self.pool(x)

      x = torch.relu(self.conv3(x))
      x = torch.relu(self.conv4(x))
      x = self.pool(x)

      x = torch.relu(self.conv5(x))
      x = torch.relu(self.conv6(x))
      x = self.pool(x)

      x = x.view(x.size(0), -1)
      x = torch.relu(self.fc1(x))
      x = torch.relu(self.fc2(x))
      x = self.fc3(x)
      return x

In [ ]:
import pickle
criterion = nn.CrossEntropyLoss()
model_1 = Lenet_5(num_classes=10).to(device)
optimizer = optim.Adam(model_1.parameters(), lr=0.001)
train_loss_list_1, metrics_1 = train_and_test(model_1, train_loader, test_loader, criterion, optimizer, 20, device)
with open('./train_loss_list_1.pkl', 'wb') as file:
    pickle.dump(train_loss_list_1, file)
with open('./metrics_1.pkl', 'wb') as file:
    pickle.dump(metrics_1, file)

[Epoch [1/20], Loss: 0.1975
[Epoch [2/20], Loss: 0.0676
[Epoch [3/20], Loss: 0.0490
[Epoch [4/20], Loss: 0.0402
[Epoch [5/20], Loss: 0.0312
[Epoch [6/20], Loss: 0.0285
[Epoch [7/20], Loss: 0.0226
[Epoch [8/20], Loss: 0.0198
[Epoch [9/20], Loss: 0.0187
[Epoch [10/20], Loss: 0.0164
[Epoch [11/20], Loss: 0.0059
[Epoch [12/20], Loss: 0.0030
[Epoch [13/20], Loss: 0.0020
[Epoch [14/20], Loss: 0.0015
[Epoch [15/20], Loss: 0.0011
[Epoch [16/20], Loss: 0.0009
[Epoch [17/20], Loss: 0.0006
[Epoch [18/20], Loss: 0.0004
[Epoch [19/20], Loss: 0.0003
[Epoch [20/20], Loss: 0.0004
Accuracy of the network : 99.11%
Precision: 1.00
Recall: 1.00
F1-score: 1.00


In [ ]:
import pickle
criterion = nn.CrossEntropyLoss()
model_2 = ResNet(BasicBlock,[2,2,2,2],num_classes=10).to(device)
optimizer = optim.Adam(model_2.parameters(), lr=0.001)
train_loss_list_2, metrics_2 = train_and_test(model_2, train_loader, test_loader, criterion, optimizer, 20, device)
with open('./train_loss_list_2.pkl', 'wb') as file:
    pickle.dump(train_loss_list_2, file)
with open('./metrics_2.pkl', 'wb') as file:
    pickle.dump(metrics_2, file)

[Epoch [1/20], Loss: 0.1113
[Epoch [2/20], Loss: 0.0527
[Epoch [3/20], Loss: 0.0383
[Epoch [4/20], Loss: 0.0329
[Epoch [5/20], Loss: 0.0255
[Epoch [6/20], Loss: 0.0216
[Epoch [7/20], Loss: 0.0169
[Epoch [8/20], Loss: 0.0140
[Epoch [9/20], Loss: 0.0121
[Epoch [10/20], Loss: 0.0101
[Epoch [11/20], Loss: 0.0033
[Epoch [12/20], Loss: 0.0016
[Epoch [13/20], Loss: 0.0010
[Epoch [14/20], Loss: 0.0009
[Epoch [15/20], Loss: 0.0005
[Epoch [16/20], Loss: 0.0003
[Epoch [17/20], Loss: 0.0002
[Epoch [18/20], Loss: 0.0001
[Epoch [19/20], Loss: 0.0001
[Epoch [20/20], Loss: 0.0001
Accuracy of the network : 99.55%
Precision: 1.00
Recall: 1.00
F1-score: 1.00


In [ ]:
import pickle
criterion = nn.CrossEntropyLoss()
model_3 = Alexnet(num_classes=10).to(device)
optimizer = optim.Adam(model_3.parameters(), lr=0.001)
train_loss_list_3, metrics_3 = train_and_test(model_3, train_loader, test_loader, criterion, optimizer, 20, device)
with open('./train_loss_list_3.pkl', 'wb') as file:
    pickle.dump(train_loss_list_3, file)
with open('./metrics_3.pkl', 'wb') as file:
    pickle.dump(metrics_3, file)

[Epoch [1/20], Loss: 0.2621
[Epoch [2/20], Loss: 0.0963
[Epoch [3/20], Loss: 0.0775
[Epoch [4/20], Loss: 0.0670
[Epoch [5/20], Loss: 0.0582
[Epoch [6/20], Loss: 0.0502
[Epoch [7/20], Loss: 0.0513
[Epoch [8/20], Loss: 0.0448
[Epoch [9/20], Loss: 0.0510
[Epoch [10/20], Loss: 0.0462
[Epoch [11/20], Loss: 0.0185
[Epoch [12/20], Loss: 0.0105
[Epoch [13/20], Loss: 0.0066
[Epoch [14/20], Loss: 0.0050
[Epoch [15/20], Loss: 0.0043
[Epoch [16/20], Loss: 0.0030
[Epoch [17/20], Loss: 0.0022
[Epoch [18/20], Loss: 0.0026
[Epoch [19/20], Loss: 0.0019
[Epoch [20/20], Loss: 0.0015
Accuracy of the network : 99.37%
Precision: 1.00
Recall: 1.00
F1-score: 1.00


In [ ]:
import pickle
criterion = nn.CrossEntropyLoss()
model_4 = VGGnet(num_classes=10).to(device)
optimizer = optim.Adam(model_4.parameters(), lr=0.001)
train_loss_list_4, metrics_4 = train_and_test(model_4, train_loader, test_loader, criterion, optimizer, 20, device)
with open('./train_loss_list_4.pkl', 'wb') as file:
    pickle.dump(train_loss_list_4, file)
with open('./metrics_4.pkl', 'wb') as file:
    pickle.dump(metrics_4, file)

[Epoch [1/20], Loss: 0.1727
[Epoch [2/20], Loss: 0.0640
[Epoch [3/20], Loss: 0.0472
[Epoch [4/20], Loss: 0.0411
[Epoch [5/20], Loss: 0.0365
[Epoch [6/20], Loss: 0.0334
[Epoch [7/20], Loss: 0.0318
[Epoch [8/20], Loss: 0.0279
[Epoch [9/20], Loss: 0.0291
[Epoch [10/20], Loss: 0.0270
[Epoch [11/20], Loss: 0.0083
[Epoch [12/20], Loss: 0.0028
[Epoch [13/20], Loss: 0.0012
[Epoch [14/20], Loss: 0.0005
[Epoch [15/20], Loss: 0.0002
[Epoch [16/20], Loss: 0.0001
[Epoch [17/20], Loss: 0.0000
[Epoch [18/20], Loss: 0.0000
[Epoch [19/20], Loss: 0.0001
[Epoch [20/20], Loss: 0.0007
Accuracy of the network : 99.42%
Precision: 1.00
Recall: 1.00
F1-score: 1.00
